### Домашняя работа №4 (Граур Андрей Константинович)

#### Шаг 1

Скачать csv файлы и создать таблицы в бд

In [1]:
import sqlite3 
from sqlite3 import Error 

import pandas as pd

In [2]:
transactions = pd.read_csv('transactions_for_dz2.csv')
transactions.head()

,TRANSACTION_ID,TX_DATETIME,CUSTOMER_ID,TERMINAL_ID,TX_AMOUNT
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66


In [3]:
clients = pd.read_csv('client_info.csv', sep=';')
clients.head()

,START_DT,END_DT,CLIENT_NAME,YEAR_BIRTH,CLIENT_ID
0,2015-07-16,2021-01-01,Olivia,1979,2213
1,2015-07-17,2021-01-01,Emma,1979,1148
2,2015-07-18,2021-01-01,Charlotte,1979,2293
3,2015-07-19,2021-01-01,Amelia,1979,1867
4,2015-07-20,2021-01-01,Ava,1979,1767


In [4]:
def create_connection(path):

    """
    Функция подключения к базе данных
    """

    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection


def sql_insert(connection, data, name, m=0, n=1):

    """
    Функция для добавления значений в таблицу
    """

    cursor = connection.cursor()
    try:
        cursor.executemany(f"""
                INSERT INTO {name} {tuple(data.columns[m:n])} 
                    VALUES ({str("?, " * (n-m-1) + "?")})""",
                data.iloc[:, m:n].values)
        connection.commit()
        print('Data inserted successfully')
    except Error as e:
        print(f"The error '{e}' occurred")

def execute_query(connection, query):

    """
    Функция выполнения запроса к базе данных
    """

    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [5]:
connection = create_connection("clients_transactions.db")
transaction_bd = """
    CREATE TABLE IF NOT EXISTS transaction_bd (
        transaction_id INTEGER PRIMARY KEY AUTOINCREMENT,
        tx_datetime NUMERIC NOT NULL,
        customer_id INTEGER NOT NULL,
        terminal_id INTEGER NOT NULL,
        tx_amount REAL NOT NULL
    );
    """

execute_query(connection, transaction_bd)

Connection to SQLite DB successful
Query executed successfully


In [6]:
customer_bd = """
    CREATE TABLE IF NOT EXISTS customer_bd (
        start_dt NUMERIC NOT NULL,
        end_dt NUMERIC NOT NULL,
        client_name TEXT NOT NULL,
        year_birth INTEGER NOT NULL,
        client_id INTEGER PRIMARY KEY
    );
    """

execute_query(connection, customer_bd)

Query executed successfully


In [7]:
sql_insert(connection, transactions, 'transaction_bd', 0, transactions.shape[1])
sql_insert(connection, clients, 'customer_bd', 0, clients.shape[1])

Data inserted successfully
Data inserted successfully


In [8]:
pd.read_sql('SELECT * FROM transaction_bd limit 5', connection)

,transaction_id,tx_datetime,customer_id,terminal_id,tx_amount
0,0,2023-01-01 00:00:31,596,3156,533.07
1,1,2023-01-01 00:02:10,4961,3412,808.56
2,2,2023-01-01 00:07:56,2,1365,1442.94
3,3,2023-01-01 00:09:29,4128,8737,620.65
4,4,2023-01-01 00:10:34,927,9906,490.66


In [9]:
pd.read_sql('SELECT * FROM customer_bd limit 5', connection)

,start_dt,end_dt,client_name,year_birth,client_id
0,2013-08-08,2999-12-31,Eleni,1972,0
1,2010-10-19,2999-12-31,Jocelyn,1954,1
2,2012-02-29,2999-12-31,Bria,1965,2
3,2017-01-09,2023-04-15,Analiah,1984,3
4,2011-05-06,2023-04-16,Kameron,1958,4


#### Шаг 2
(Балл - 2 за каждый пункт) Написать скрипты:

a. Вывести список всех клиентов, у которых между двумя ближайшими транзакциями был перерыв больше 35 дней хотя бы один раз (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [12]:
pd.read_sql(
"""
    WITH transactions AS (
        SELECT customer_id, tx_datetime,
               LAG(tx_datetime) OVER (PARTITION BY customer_id ORDER BY tx_datetime) AS prev_tx_datetime
        FROM transaction_bd
    ),
    span AS (
        SELECT customer_id, (julianday(tx_datetime) - julianday(prev_tx_datetime)) AS days
        FROM transactions
        WHERE prev_tx_datetime IS NOT NULL
    ),
    customers_span AS (
        SELECT customer_id, days
        FROM span
        WHERE days > 35
    )
    SELECT cbd.client_name, cbd.year_birth, cbd.client_id, cs.days
    FROM customer_bd cbd
    JOIN customers_span cs ON cbd.client_id = cs.customer_id
    WHERE julianday('2023-05-01') <= julianday(cbd.end_dt);
""", connection)

,client_name,year_birth,client_id,days
0,Kiarra,2000,24,40.150590
1,Vaishnavi,2000,480,54.370278
2,Malayna,2000,639,35.321551
3,Becca,1992,707,35.116146
4,Avionna,1998,812,82.998032
5,Macarena,2000,896,50.484167
6,Kavya,1984,1129,41.842755
7,Nalah,1986,1299,65.272905
8,Blakley,1985,1334,55.791875
9,Nalia,1996,1459,44.903495


b. Вывести список клиентов, у которых максимальная сумма транзакции (имеется ввиду максимальное значение TX_AMOUNT) больше 50000. В ответе должен быть id клиента, максимальное значение суммы транзакций, минимальное значение суммы транзакций (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).


In [13]:
pd.read_sql(
"""
    WITH transactions AS (
        SELECT customer_id,
               MAX(tx_amount) AS max_tr_sum,
               MIN(tx_amount) AS min_tr_sum
        FROM transaction_bd
        GROUP BY customer_id
    ),
    clients AS (
        SELECT customer_id, max_tr_sum, min_tr_sum
        FROM transactions
        WHERE max_tr_sum > 50000
    )
    SELECT cl.customer_id, cl.max_tr_sum, cl.min_tr_sum
    FROM customer_bd c
    JOIN clients cl ON c.client_id = cl.customer_id
    WHERE julianday('now') <= julianday(c.end_dt);
""", connection)

,customer_id,max_tr_sum,min_tr_sum
0,3013,77212.50,1.46
1,3494,53213.00,85.76
2,4253,51937.25,37.95


c. Вывести топ-10 клиентов (сортируя их по убыванию id), у которых сумма дневных транзакций больше суммы вечерних транзакций Условимся, что дневными считаются транзакции, которые были произведены до 12 часов, а вечерними - после 12 часов включительно. (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [14]:
pd.read_sql(
"""
    WITH transactions AS (
        SELECT customer_id,
               SUM(CASE WHEN strftime('%H', tx_datetime) <= '11:59:59' THEN tx_amount ELSE 0 END) AS tr_sum_daily,
               SUM(CASE WHEN strftime('%H', tx_datetime) > '11:59:59' THEN tx_amount ELSE 0 END) AS tr_sum_at_night
        FROM transaction_bd
        GROUP BY customer_id
    ),
    clients AS (
        SELECT customer_id
        FROM transactions
        WHERE tr_sum_daily > tr_sum_at_night
    )
    SELECT c.client_id
    FROM customer_bd c
    JOIN clients cl ON c.client_id = cl.customer_id
    WHERE julianday('2023-05-01') <= julianday(c.end_dt)
    ORDER BY c.client_id DESC
    LIMIT 10;
""", connection)

,client_id
0,4997
1,4993
2,4990
3,4987
4,4983
5,4982
6,4978
7,4976
8,4973
9,4972


d. Найти для каждого клиента день, когда у него была максимальная сумма транзакции, то есть вывести клиента, дату транзакции, сумму транзакции (клиент должен быть действующим, то есть дата закрытия записи о клиенте не должна быть меньше дня относительно которого мы считаемся).

In [15]:
pd.read_sql(
"""
    WITH transactions AS (
        SELECT customer_id, MAX(tx_amount) AS max_tr_amount
        FROM transaction_bd
        GROUP BY customer_id
    ),
    transactions_by_date AS (
        SELECT t.customer_id, t.tx_datetime, t.tx_amount
        FROM transaction_bd t
        JOIN transactions tr ON t.customer_id = tr.customer_id AND t.tx_amount = tr.max_tr_amount
    )
    SELECT c.client_id, tbd.tx_datetime, tbd.tx_amount
    FROM customer_bd c
    JOIN transactions_by_date tbd ON c.client_id = tbd.customer_id
    WHERE julianday('2023-05-01') <= julianday(c.end_dt)
    ORDER BY c.client_id;
""", connection)

,client_id,tx_datetime,tx_amount
0,0,2023-02-01 05:30:38,1258.86
1,1,2023-02-07 18:42:49,17152.00
2,2,2023-02-21 20:09:34,1720.11
3,5,2023-03-21 13:00:37,1156.11
4,6,2023-03-20 13:13:07,369.26
...,...,...,...
3731,4991,2023-01-25 15:29:38,1070.26
3732,4992,2023-03-10 19:51:24,839.03
3733,4993,2023-01-13 15:50:46,1054.57
3734,4996,2023-01-11 09:47:27,224.62


e. Вычислить интегральное распределение суммы транзакций для каждого дня за весь период, то есть вывести дату транзакции, сумму транзакции и интегральное распределение по сумме (не учитываем что клиент может быть недействующим)

In [16]:
pd.read_sql(
"""
    WITH transactions AS (
        SELECT DATE(tx_datetime) AS tr_date, SUM(tx_amount) AS daily_sum
        FROM transaction_bd
        GROUP BY DATE(tx_datetime)
    ),
    summarized AS (
        SELECT daily_sum,
               tr_date,
               SUM(daily_sum) OVER (ORDER BY tr_date) AS tr_sum
        FROM transactions
    )
    SELECT tr_date, daily_sum, tr_sum
    FROM summarized
    ORDER BY tr_date;
""", connection)

,tr_date,daily_sum,tr_sum
0,2023-01-01,4827656.26,4.827656e+06
1,2023-01-02,4862551.41,9.690208e+06
2,2023-01-03,5058973.71,1.474918e+07
3,2023-01-04,4938142.47,1.968732e+07
4,2023-01-05,5002954.23,2.469028e+07
...,...,...,...
105,2023-04-16,5299386.81,5.469860e+08
106,2023-04-17,5100973.31,5.520870e+08
107,2023-04-18,5233557.39,5.573205e+08
108,2023-04-19,5194846.44,5.625154e+08


In [17]:
connection.close()